In [1]:
'''
This is to demonstrate a vector to a random direction on a map

1. Spawn a car
2. Pick a point on the map
3. Draw a route to the point
4. Get a vector and spin the car to face that point

'''

#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import random
import sys
import math

In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000)

In [8]:
#client.load_world('Town03')

#define environment/world and get possible places to spawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*model3*')

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
#setting RGB Camera - this follow the approach explained in a Carla video
# link: https://www.youtube.com/watch?v=om8klsBj4rc&t=1184s


In [6]:
spectator = world.get_spectator()
vehicle_transform = vehicle.get_transform()
spectator_pos = carla.Transform(vehicle_transform.location + carla.Location(x=0,y=0,z=150),
                                carla.Rotation(pitch = vehicle_transform.rotation.pitch -90))
spectator.set_transform(spectator_pos)



In [4]:
# get a point on the amp and draw a route

# route planning bit like shown in Tutorial 3
sys.path.append('C:/CARLA_0.9.15/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

point_a = start_point.location #we start at where the car is

sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

# now let' pick the longest possible route
distance = 0
for loc in spawn_points: # we start trying all spawn points 
                            #but we just exclude first at zero index
    cur_route = grp.trace_route(point_a, loc.location)
    if len(cur_route)>distance:
        distance = len(cur_route)
        route = cur_route
#draw the route in sim window - Note it does not get into the camera of the car
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=600.0,
        persistent_lines=True)
    
destination = route[-1][0]

In [6]:
#getting location of the end of the route
destination = route[-1][0]

In [33]:
#example to get angle
print(get_angle(vehicle,destination))

9.092582599734246


In [9]:
# showing a car driving around and displaying angle to destination


CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 


#function from GPS tutorials
def get_angle(car,wp):
    '''
    this function returns degrees between the car's direction 
    and direction to a selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees<-180:
        degrees = degrees + 360
    elif degrees > 180:
        degrees = degrees - 360
    return degrees

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1


camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') 
camera_bp.set_attribute('image_size_y', '480')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

vehicle.set_autopilot(True)

#main loop 
quit = False

while True:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        vehicle.set_autopilot(False)
        vehicle.apply_control(carla.VehicleControl(throttle=0, 
                                                   steer=0,
                                                  brake=1))
        quit = True
        break
    image = camera_data['image']
    
    #get current speed
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    # now we add the speed to the window showing a camera mounted on the car
    image = cv2.putText(image, 'Speed: '+str(int(speed))+' kmh', org2, 
                        font, fontScale, color, thickness, cv2.LINE_AA)

    # show forward vector values (z dimension is ignored as we are not navigating up or down)
    vector_to_destination = get_angle(vehicle,destination)
    distance_to_destination = vehicle.get_transform().location.distance(destination.transform.location)
    # now we overlay x and y
    image = cv2.putText(image, 'Angle to destination: '+str(round(vector_to_destination,3)), org3, 
                        font, fontScale, color, thickness, cv2.LINE_AA)
    image = cv2.putText(image, 'Distance to destination: '+str(round(distance_to_destination,3)), org4, 
                        font, fontScale, color, thickness, cv2.LINE_AA)
    
    cv2.imshow('RGB Camera',image)

#clean up
cv2.destroyAllWindows()
camera.stop()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

In [1]:
destination.location

NameError: name 'destination' is not defined

9.092582599734246


In [18]:
initial_yaw = vehicle_transform.rotation.yaw

for i in range(50):
    angle_adj = random.randrange(-30, 30, 1)
    vehicle_transform.rotation.yaw = initial_yaw +angle_adj 
    vehicle.set_transform(vehicle_transform)
    time.sleep(1)

In [7]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
